In [1]:
import numpy as np
from scipy import linalg as la
import networkx as nx
import itertools
import regex as re
# Problems 1-2
class DiGraph:
    """A class for representing directed graphs via their adjacency matrices.

    Attributes:
        (fill this out after completing DiGraph.__init__().)
    """
    # Problem 1
    def __init__(self, A, labels=None):
        """Modify A so that there are no sinks in the corresponding graph,
        then calculate Ahat. Save Ahat and the labels as attributes.

        Parameters:
            A ((n,n) ndarray): the adjacency matrix of a directed graph.
                A[i,j] is the weight of the edge from node j to node i.
            labels (list(str)): labels for the n nodes in the graph.
                If None, defaults to [0, 1, ..., n-1].
        """

        if labels != None:
            if len(labels) != A.shape[0]:
                raise ValueError("Number of labels must equal number of nodes")   #check for errors
        
        m,n = A.shape

        for j in range(n):        #make any sink column into a column of ones
            g = 0
            for entry in A[:,j]:
                if entry == 0:
                    g += 1
            if g == n:
                for z in range(m):
                    A[z, j] = 1

        col_sums = np.sum(A, axis=0)       #get the sum of each column

        if labels is None:      #labels default to the given array
            labels = [i for i in range(n)]

        A = A / col_sums     #normalize each column of A

        self.labels = labels
        self.ahat = A

        

    # Problem 2
    def linsolve(self, epsilon=0.85):
        """Compute the PageRank vector using the linear system method.

        Parameters:
            epsilon (float): the damping factor, between 0 and 1.

        Returns:
            dict(str -> float): A dictionary mapping labels to PageRank values.
        """
        m,n = self.ahat.shape  
        A = self.ahat       #get A hat
        I = np.identity(n)
        one = np.ones(n)         #define the vectors and matrices that we need to solve the system

        p = la.solve(I - epsilon*A, ((1-epsilon)/n) * one)      #solve the system
        
        final_dict = dict()

        for i, label in enumerate(self.labels):    #create our dictionary
            final_dict[label] = p[i]

        return final_dict

    # Problem 2
    def eigensolve(self, epsilon=0.85):
        """Compute the PageRank vector using the eigenvalue method.
        Normalize the resulting eigenvector so its entries sum to 1.

        Parameters:
            epsilon (float): the damping factor, between 0 and 1.

        Return:
            dict(str -> float): A dictionary mapping labels to PageRank values.
        """
        A = self.ahat
        m,n = A.shape            #define the matrices we will need for our eigenvector solving
        E = np.ones((n,n))

        B = epsilon * A +((1-epsilon)/n) * E       #define B

        p = la.eig(B)[1][:,0] / np.sum(la.eig(B)[1][:,0])      #get the largest normalized eigenvector of B

        final_dict = dict()

        for i, label in enumerate(self.labels):    #create our dictionary
            final_dict[label] = p[i]

        return final_dict

    # Problem 2
    def itersolve(self, epsilon=0.85, maxiter=100, tol=1e-12):
        """Compute the PageRank vector using the iterative method.

        Parameters:
            epsilon (float): the damping factor, between 0 and 1.
            maxiter (int): the maximum number of iterations to compute.
            tol (float): the convergence tolerance.

        Return:
            dict(str -> float): A dictionary mapping labels to PageRank values.
        """
        A = self.ahat
        m,n = A.shape       #create the necessary vectors and matrices
        ones = np.ones(n)

        p_0 = np.array([1/n for k in range(n)])       #create our initial guess

        for i in range(maxiter):
            p_1 = epsilon*A@p_0 + ((1-epsilon)/n)*ones
            if la.norm(p_1 - p_0, ord=1) < tol:                 #use the iterative technique until we are close enough or use max iterations
                break
            p_0 = p_1
        
        final_dict = dict()

        for i, label in enumerate(self.labels):    #create our dictionary
            final_dict[label] = p_1[i]

        return final_dict


# Problem 3
def get_ranks(d):
    """Construct a sorted list of labels based on the PageRank vector.

    Parameters:
        d (dict(str -> float)): a dictionary mapping labels to PageRank values.

    Returns:
        (list) the keys of d, sorted by PageRank value from greatest to least.
    """
    l = [(-val, key) for key, val in zip(d.keys(), d.values())]        #get the keys and values from the dictionary and create a list

    l.sort()                   #sort the the list

    return[i[1] for i in l]                 #return the keys of d from greatest to least



# Problem 4
def rank_websites(filename="web_stanford.txt", epsilon=0.85):
    """Read the specified file and construct a graph where node j points to
    node i if webpage j has a hyperlink to webpage i. Use the DiGraph class
    and its itersolve() method to compute the PageRank values of the webpages,
    then rank them with get_ranks(). If two webpages have the same rank,
    resolve ties by listing the webpage with the larger ID number first.

    Each line of the file has the format
        a/b/c/d/e/f...
    meaning the webpage with ID 'a' has hyperlinks to the webpages with IDs
    'b', 'c', 'd', and so on.

    Parameters:
        filename (str): the file to read from.
        epsilon (float): the damping factor, between 0 and 1.

    Returns:
        (list(str)): The ranked list of webpage IDs.
    """

    pattern = re.compile(r"([\d]+)[\/|\n]", re.MULTILINE)             #create a regex compiler to parse through the data

    with open(filename, 'r') as myfile:
        data1 = myfile.read()                             #open the file and read the data and establish a set of sites

    sites = pattern.findall(data1)
    sites = set(sites)

    A = np.zeros((len(sites), len(sites)))         #initialize our adjacency matrix

    siteslist = list(sites)
    siteslist = list(np.sort(siteslist))            

    with open(filename, 'r') as myfile:
        data2 = myfile.readlines()

    for line in data2:
        path = pattern.findall(line)
        index = siteslist.index(path[0])          #separate the webpage ID's and check to see if there exists a link

        for link in path[1:]:
            A[siteslist.index(link), index] += 1           #add to adjacency matrix if link exists

    graph = DiGraph(A, siteslist)
    ranks = graph.itersolve(epsilon)

    return get_ranks(ranks)



    
# Problem 5
def rank_ncaa_teams(filename, epsilon=0.85):
    """Read the specified file and construct a graph where node j points to
    node i with weight w if team j was defeated by team i in w games. Use the
    DiGraph class and its itersolve() method to compute the PageRank values of
    the teams, then rank them with get_ranks().

    Each line of the file has the format
        A,B
    meaning team A defeated team B.

    Parameters:
        filename (str): the name of the data file to read.
        epsilon (float): the damping factor, between 0 and 1.

    Returns:
        (list(str)): The ranked list of team names.
    """
    teams = set()
    games = []

    with open(filename, 'r') as gamefile:
        gamefile.readline()             #read in the games and sort the teams into set
        for line in gamefile:
            game = line.strip().split(',')
            games.append(game)
            teams.add(game[1])
            teams.add(game[0])

    teams = list(teams)
    n = len(teams)
    team_dict = dict(zip(teams, np.arange(n)))            #create a dictionary of the teams
    A = np.zeros((n,n))

    for fight in games:
        A[team_dict[fight[0]], team_dict[fight[1]]] = A[team_dict[fight[0]], team_dict[fight[1]]] + 1     #add 1 to the correct entry when a team beats the corresponding team

    
    graph = DiGraph(A, teams)
    ranks = graph.itersolve(epsilon)          #create a DiGraph object out of our results and user itersolve and get_ranks to get our final ranks
    return get_ranks(ranks)


# Problem 6
def rank_actors(filename="top250movies.txt", epsilon=0.85):
    """Read the specified file and construct a graph where node a points to
    node b with weight w if actor a and actor b were in w movies together but
    actor b was listed first. Use NetworkX to compute the PageRank values of
    the actors, then rank them with get_ranks().

    Each line of the file has the format
        title/actor1/actor2/actor3/...
    meaning actor2 and actor3 should each have an edge pointing to actor1,
    and actor3 should have an edge pointing to actor2.
    """

    DG = nx.DiGraph()           #create a directed graph
    pattern = re.compile(r"(.*)\n")        #create regex compiler to recognize pattern

    with open(filename, 'r', encoding="utf-8") as movfile:           #open file
        data = movfile.readlines()
    
    for line in data:                          #format the data
        line = line.split('/')
        line[-1] = pattern.sub(r"\1", line[-1])
        actors = line[1:]
        pairs = itertools.combinations(actors, 2)

        for ac1, ac2 in pairs:                          #add an edge between actors or add to an existing adge
            if DG.has_edge(ac2, ac1):
                DG[ac2][ac1]["weight"] += 1
            else:
                DG.add_edge(ac2, ac1, weight = 1)

    return get_ranks(nx.pagerank(DG, alpha=epsilon))

In [3]:
def test1():
    test_g = DiGraph(np.array([[0,0,0,0], [1,0,1,0], [1,0,0,1], [1,0,1,0]]))
    print(test_g.ahat)
test1()

[[0.         0.25       0.         0.        ]
 [0.33333333 0.25       0.5        0.        ]
 [0.33333333 0.25       0.         1.        ]
 [0.33333333 0.25       0.5        0.        ]]


In [6]:
def test2():
    labels = ['a', 'b', 'c', 'd']
    test_g = DiGraph(np.array([[0,0,0,0], [1,0,1,0], [1,0,0,1], [1,0,1,0]]), labels)
    d = test_g.linsolve()
    print(test_g.linsolve())
    print(test_g.eigensolve())
    print(test_g.itersolve())
    print(get_ranks(d))
test2()

{'a': 0.09575863576738085, 'b': 0.27415828596414515, 'c': 0.35592479230432883, 'd': 0.27415828596414515}
{'a': 0.09575863576738071, 'b': 0.27415828596414515, 'c': 0.3559247923043288, 'd': 0.27415828596414527}
{'a': 0.09575863576740319, 'b': 0.27415828596408354, 'c': 0.35592479230442975, 'd': 0.27415828596408354}
['c', 'b', 'd', 'a']


In [9]:
def test4():
    print(rank_websites())
test4()

['98595', '32791', '28392', '77323', '92715', '26083', '130094', '99464', '12846', '106064', '332', '31328', '86049', '123900', '74923', '119538', '90571', '116900', '139197', '114623', '15672', '20283', '108608', '6213', '136623', '56800', '62259', '67827', '64104', '96254', '82752', '203109', '178606', '24083', '217557', '68912', '41471', '203100', '19894', '177473', '203696', '121480', '268900', '246911', '146603', '10984', '84478', '182230', '210258', '110923', '147255', '185201', '102329', '178003', '188707', '197440', '208254', '228036', '247003', '199855', '230247', '112786', '110520', '121418', '101455', '41677', '66498', '63712', '84094', '88621', '35759', '61413', '55398', '17101', '32789', '51075', '21554', '72982', '104202', '1662', '94657', '4889', '48096', '81525', '11433', '26918', '79322', '237162', '132956', '79383', '264536', '31597', '76236', '209406', '215281', '253851', '6886', '72430', '181789', '218365', '71480', '26441', '78258', '26121', '260233', '159242', '64

In [8]:
def test6():
    print(rank_actors(filename="top250movies.txt", epsilon=0.39))
test6()

['Leonardo DiCaprio', 'Robert De Niro', 'Tom Hanks', 'Morgan Freeman', 'Al Pacino', 'Christian Bale', 'Harrison Ford', 'Tom Hardy', 'Clint Eastwood', 'Matt Damon', 'Ralph Fiennes', 'Ben Kingsley', 'Gary Oldman', 'Michael Caine', 'James Stewart', 'Kevin Spacey', 'Charles Chaplin', 'Brad Pitt', 'Hugh Jackman', 'Liam Neeson', 'Bruce Willis', 'F. Murray Abraham', 'Christoph Waltz', 'Aamir Khan', 'Ryan Gosling', 'Thomas Mitchell', 'Jamie Foxx', 'Martin Sheen', 'Joe Pesci', 'Jack Nicholson', 'Robert Duvall', 'Mark Hamill', 'Mark Ruffalo', 'Alec Guinness', 'Marlon Brando', 'Ben Affleck', 'Eli Wallach', 'Joseph Gordon-Levitt', 'William Holden', 'Russell Crowe', 'Daniel Day-Lewis', 'Emma Stone', 'Paul Newman', 'Faye Dunaway', 'Matthew Modine', 'Ryûnosuke Kamiki', 'Claude Rains', 'Adrien Brody', 'Jonah Hill', 'Chris Hemsworth', 'Hugo Weaving', 'Humphrey Bogart', 'Sumi Shimamoto', 'John Hurt', 'Daniel Brühl', 'Carrie Fisher', 'Matthew McConaughey', "Ryan O'Neal", 'Darsheel Safary', 'Christopher W